# Momentum strategy
This strategy consists of ranking the SP500 stocks according to their performance up to a given date, and invest by going long the best performing ones and short the ones with the worst performance.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import datetime as dt
from dateutil.relativedelta import relativedelta

import yfinance as yf
import pandas_ta as ta
import mplfinance as mpf
#import pandas_datareader.data as web
%run functions.ipynb

In [2]:
tickers=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers=tickers.Symbol.to_list()
#tickers=tickers[:10]
start='2021-01-01'
end='2023-01-01'
#end=dt.date.today()
#start=end-relativedelta(years=1)
simulation_end=pd.to_datetime(end)-relativedelta(years=1)

In [3]:
df=yf.download(tickers=tickers+['SPY'], start=start, end=end)

[*********************100%%**********************]  506 of 506 completed

6 Failed downloads:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
['KVUE', 'SOLV', 'GEV', 'VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 1609477200, endDate = 1672549200")
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2021-01-01 -> 2023-01-01)')


In [4]:
idx=pd.IndexSlice
stocks=df.stack(level=1).reorder_levels([1,0])
stocks.index=stocks.index.set_names(['Symbol','Date'])

spy=stocks.loc[idx['SPY',:]]
stocks=stocks.drop('SPY',level=0,axis=0)

In [5]:
close=stocks.Close
close.sort_index(level=[0,1])
close=close.reorder_levels([1,0])
close_past=close.loc[idx[:simulation_end,:]]  #reorder because loc doesn't work on second level of multiindex
close_now=close.loc[idx[simulation_end:,:]]

#monthly returns in the past
monthly_stock_past=close_past.groupby(['Symbol', pd.Grouper(freq='M',level=0)])\
                         .apply(lambda x: np.exp(log_ret(x).sum()))


#average monthly return for each asset, in the past period
monthly_stock_past=pd.DataFrame(monthly_stock_past)
avg_stock_past=monthly_stock_past.Close.groupby('Symbol').apply(lambda x: np.exp(np.log(x).mean()))
avg_stock_past=avg_stock_past.to_frame()


#Divide stocks into best and worst according to average monthly return
quantile=10
avg_stock_past['Quantile']=pd.qcut(avg_stock_past.squeeze(),quantile,labels=False)

winners=avg_stock_past[avg_stock_past.Quantile==quantile-1].Close
losers=avg_stock_past[avg_stock_past.Quantile==0].Close



#returns of an equally weighted portfolio of each of the two groups
winners_past_ret=winners.mean()
losers_past_ret=losers.mean()

#total return in the past period
total_past_ret=1+(winners_past_ret-1)-(losers_past_ret-1)




#Compute average monthly return of the strategy on the new (test) period:

winners_list=winners.index.get_level_values(level=0).to_list()
losers_list =losers.index.get_level_values(level=0).to_list()

winners_ret=close.loc[idx[simulation_end:,winners_list]]\
            .groupby(['Symbol',pd.Grouper(freq='M',level=0)])\
            .apply(lambda x: np.exp(log_ret(x).sum()))\
            .mean()

losers_ret=close.loc[idx[simulation_end:,losers_list]]\
            .groupby('Symbol')\
            .apply(lambda x: np.exp(log_ret(x).sum()))\
            .mean()

total_ret=1+winners_ret-losers_ret




#Benchmark: average monthly return of SP500
#monthly returns for SP500
spy_mo_old=spy.Close[:simulation_end].resample('M').apply(lambda x: np.exp(log_ret(x).sum()))
spy_mo_new=spy.Close[simulation_end:].resample('M').apply(lambda x: np.exp(log_ret(x).sum()))
#average monthly return for SP500
spy_avg_old=np.exp(np.log(spy_mo_old).mean())
spy_avg_new=np.exp(np.log(spy_mo_new).mean())


In [6]:
print('past strategy return: ', round(total_past_ret, 3))
print('past benchmark return: ', round(spy_avg_old,3))
print('------------------------------------------')
print('test strategy return: ', round(total_ret, 3))
print('test benchmark return: ', round(spy_avg_new,3))


past strategy return:  1.063
past benchmark return:  1.016
------------------------------------------
test strategy return:  1.104
test benchmark return:  0.98
